In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
! kaggle datasets download -d virajbagal/roco-dataset

Dataset URL: https://www.kaggle.com/datasets/virajbagal/roco-dataset
License(s): CC0-1.0
100% 6.17G/6.19G [00:48<00:00, 150MB/s]
100% 6.19G/6.19G [00:49<00:00, 136MB/s]


In [ ]:
!unzip roco-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: all_data/validation/radiology/images/PMC3870636_CRIM.OTOLARYNGOLOGY2013-650428.002.jpg  
  inflating: all_data/validation/radiology/images/PMC3870648_CRIM.DENTISTRY2013-378062.012.jpg  
  inflating: all_data/validation/radiology/images/PMC3871037_enm-28-326-g001.jpg  
  inflating: all_data/validation/radiology/images/PMC3872022_CRIM.MEDICINE2013-653925.001.jpg  
  inflating: all_data/validation/radiology/images/PMC3872161_CRIM.SURGERY2013-209494.001.jpg  
  inflating: all_data/validation/radiology/images/PMC3872390_CRIM.OBGYN2013-906351.001.jpg  
  inflating: all_data/validation/radiology/images/PMC3872571_OJO-6-193-g005.jpg  
  inflating: all_data/validation/radiology/images/PMC3872571_OJO-6-193-g007.jpg  
  inflating: all_data/validation/radiology/images/PMC3872649_SNI-4-150-g001.jpg  
  inflating: all_data/validation/radiology/images/PMC3872649_SNI-4-150-g014.jpg  
  inflating: all_data/validation/radiology/images/PMC38

LSTM

In [ ]:
!pip install faker
from faker import Faker
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

In [ ]:
!pip install faker
from faker import Faker
import random
from tensorflow.keras.preprocessing.text import Tokenizer

captions_path = '/content/all_data/train/non-radiology/captions.txt'

with open(captions_path, 'r') as file:
    captions = file.readlines()

texts = []
labels = []


In [ ]:
import random

num_samples = len(captions)
texts = []
labels = []

for idx, line in enumerate(captions):
    if random.random() < 0.5:
        texts.append(line.strip())
        labels.append(1)
    else:
        texts.append(line.strip())
        labels.append(0)
    if idx % 1000 == 0:
        print(f"Processed {idx} / {num_samples} captions...")

print("Completed processing captions.")

split_index = int(0.8 * len(texts))
train_texts = texts[:split_index]
test_texts = texts[split_index:]
train_labels = labels[:split_index]
test_labels = labels[split_index:]

print(f"Number of training samples: {len(train_texts)}")
print(f"Number of testing samples: {len(test_texts)}")
print(f"Sample training text: {train_texts[0]}")
print(f"Sample label: {train_labels[0]}")


Processed 0 / 4895 captions...
Processed 1000 / 4895 captions...
Processed 2000 / 4895 captions...
Processed 3000 / 4895 captions...
Processed 4000 / 4895 captions...
Completed processing captions.
Number of training samples: 3916
Number of testing samples: 979
Sample training text: ROCO_81826	 Model bone showing the extent of graft insertion
Sample label: 0


In [ ]:

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Display some basic stats
print("Number of training samples:", len(train_sequences))
print("Number of test samples:", len(test_sequences))
print("Vocabulary size:", len(tokenizer.word_index) + 1)

Number of training samples: 3916
Number of test samples: 979
Vocabulary size: 18851


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_texts)

In [ ]:
train_sequences=tokenizer.texts_to_sequences(train_texts)
test_sequences=tokenizer.texts_to_sequences(test_texts)

In [ ]:
train_sequences

[[3, 6943, 95, 73, 17, 1, 723, 2, 187, 417],
 [3, 6944, 1536, 1390, 3446, 43],
 [3, 6945, 758, 2, 844, 6946, 76, 7, 6947, 1, 188, 488],
 [3, 6948, 246, 286, 2, 1, 272, 36, 680, 6949, 6950, 4554],
 [3,
  6951,
  350,
  7,
  6952,
  4,
  6953,
  1,
  103,
  548,
  2,
  4555,
  1260,
  759,
  21,
  115,
  6,
  1,
  65,
  14,
  311,
  287,
  22,
  351,
  2,
  1,
  29,
  147,
  156,
  16,
  3447,
  6954,
  3448,
  6,
  1,
  55,
  2,
  1,
  2729,
  4556,
  6955,
  1,
  1716,
  4,
  2729,
  4556,
  9,
  5,
  1127,
  549,
  50,
  1934,
  6956,
  6957,
  6958,
  4,
  3449,
  6,
  5,
  2261,
  87,
  1537,
  30,
  1717,
  16,
  1261,
  6,
  4557,
  4,
  2262,
  2730],
 [3, 6959, 489, 2731, 6, 352, 2, 2263, 4558, 4, 6960, 74, 25],
 [3, 6961, 5, 6962, 119, 26, 260, 34, 7, 1, 29, 1935, 112, 39, 845, 112],
 [3, 6963, 6964, 6965, 312],
 [3,
  6966,
  417,
  2,
  5,
  1391,
  6,
  1,
  20,
  2732,
  122,
  2733,
  1128,
  1,
  15,
  37,
  1,
  61,
  2,
  1,
  2733,
  796,
  4,
  233,
  8,
  2264,
  7,


In [ ]:
word_index=tokenizer.word_index
max_sequence_length=max(len(seq) for seq in train_sequences)
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length

240

In [ ]:
train_padded=pad_sequences(train_sequences,maxlen=max_sequence_length,padding="post")
test_padded=pad_sequences(test_sequences,maxlen=max_sequence_length,padding="post")
test_padded

array([[   3,  631, 6859, ...,    0,    0,    0],
       [   3,    1, 2823, ...,    0,    0,    0],
       [   3,  847,  379, ...,    0,    0,    0],
       ...,
       [   3,   41, 2247, ...,    0,    0,    0],
       [   3,  241,  154, ...,    0,    0,    0],
       [   3, 2124,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

In [ ]:
train_labels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [ ]:
from gensim.models import Word2Vec
w2v_model=Word2Vec(sentences=[text.split() for text in train_texts],vector_size=100)

In [ ]:
import numpy as np
embedding_dim=100
embedding_matrix=np.zeros((len(word_index)+1,embedding_dim))
for word,i in word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i]=w2v_model.wv[word]
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.46336579,  0.57862461,  0.03903345, ..., -0.55314571,
        -0.02143799,  0.24291405],
       [-0.52000338,  0.64166498,  0.04603353, ..., -0.58050609,
        -0.03678184,  0.26509756],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [ ]:
model=Sequential([
        Embedding(input_dim=len(word_index)+1,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
        input_length=max_sequence_length,
        trainable=False),
        LSTM(128),
        Dense(2,activation="softmax")
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_padded,train_labels,epochs=10,batch_size=32,validation_split=0.2)

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 383ms/step - accuracy: 0.5009 - loss: 0.6935 - val_accuracy: 0.5217 - val_loss: 0.6926
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 42s 397ms/step - accuracy: 0.5282 - loss: 0.6925 - val_accuracy: 0.5230 - val_loss: 0.6928
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 37s 355ms/step - accuracy: 0.5123 - loss: 0.6932 - val_accuracy: 0.5217 - val_loss: 0.6925
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 352ms/step - accuracy: 0.5091 - loss: 0.6931 - val_accuracy: 0.5217 - val_loss: 0.6926
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 424ms/step - accuracy: 0.5104 - loss: 0.6931 - val_accuracy: 0.5217 - val_loss: 0.6927
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 77s 376ms/step - accuracy: 0.5063 - loss: 0.6932 - val_accuracy: 0.5217 - val_loss: 0.6926
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 40s 367ms/step - accuracy: 0.5164 - loss: 0.6928 - val_accuracy: 0.5217 - val_loss: 0.6927
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 42s 376ms/step - accuracy: 0.5056 - loss: 0.6932 - val_accu

In [ ]:
test_loss,test_accuracy=model.evaluate(test_padded,test_labels)

31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 154ms/step - accuracy: 0.4456 - loss: 0.6953


In [ ]:
test_accuracy

0.46271705627441406

Bi-Directional LSTM

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [ ]:
import random

num_samples = len(captions)
texts = []
labels = []

for idx, line in enumerate(captions):
    if random.random() < 0.5:
        texts.append(line.strip())
        labels.append(1)
    else:
        texts.append(line.strip())
        labels.append(0)
    if idx % 1000 == 0:
        print(f"Processed {idx} / {num_samples} captions...")

print("Completed processing captions.")

split_index = int(0.8 * len(texts))
train_texts = texts[:split_index]
test_texts = texts[split_index:]
train_labels = labels[:split_index]
test_labels = labels[split_index:]

print(f"Number of training samples: {len(train_texts)}")
print(f"Number of testing samples: {len(test_texts)}")
print(f"Sample training text: {train_texts[0]}")
print(f"Sample label: {train_labels[0]}")


Processed 0 / 4895 captions...
Processed 1000 / 4895 captions...
Processed 2000 / 4895 captions...
Processed 3000 / 4895 captions...
Processed 4000 / 4895 captions...
Completed processing captions.
Number of training samples: 3916
Number of testing samples: 979
Sample training text: ROCO_81826	 Model bone showing the extent of graft insertion
Sample label: 1


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences=tokenizer.texts_to_sequences(train_texts)
test_sequences=tokenizer.texts_to_sequences(test_texts)
train_sequences

[[3, 6943, 95, 73, 17, 1, 723, 2, 187, 417],
 [3, 6944, 1536, 1390, 3446, 43],
 [3, 6945, 758, 2, 844, 6946, 76, 7, 6947, 1, 188, 488],
 [3, 6948, 246, 286, 2, 1, 272, 36, 680, 6949, 6950, 4554],
 [3,
  6951,
  350,
  7,
  6952,
  4,
  6953,
  1,
  103,
  548,
  2,
  4555,
  1260,
  759,
  21,
  115,
  6,
  1,
  65,
  14,
  311,
  287,
  22,
  351,
  2,
  1,
  29,
  147,
  156,
  16,
  3447,
  6954,
  3448,
  6,
  1,
  55,
  2,
  1,
  2729,
  4556,
  6955,
  1,
  1716,
  4,
  2729,
  4556,
  9,
  5,
  1127,
  549,
  50,
  1934,
  6956,
  6957,
  6958,
  4,
  3449,
  6,
  5,
  2261,
  87,
  1537,
  30,
  1717,
  16,
  1261,
  6,
  4557,
  4,
  2262,
  2730],
 [3, 6959, 489, 2731, 6, 352, 2, 2263, 4558, 4, 6960, 74, 25],
 [3, 6961, 5, 6962, 119, 26, 260, 34, 7, 1, 29, 1935, 112, 39, 845, 112],
 [3, 6963, 6964, 6965, 312],
 [3,
  6966,
  417,
  2,
  5,
  1391,
  6,
  1,
  20,
  2732,
  122,
  2733,
  1128,
  1,
  15,
  37,
  1,
  61,
  2,
  1,
  2733,
  796,
  4,
  233,
  8,
  2264,
  7,


In [ ]:
word_index=tokenizer.word_index
max_sequence_length=max(len(seq) for seq in train_sequences)
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length
train_padded=pad_sequences(train_sequences,maxlen=max_sequence_length,padding="post")
test_padded=pad_sequences(test_sequences,maxlen=max_sequence_length,padding="post")
test_padded

array([[   3,  631, 6859, ...,    0,    0,    0],
       [   3,    1, 2823, ...,    0,    0,    0],
       [   3,  847,  379, ...,    0,    0,    0],
       ...,
       [   3,   41, 2247, ...,    0,    0,    0],
       [   3,  241,  154, ...,    0,    0,    0],
       [   3, 2124,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)
train_labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
from gensim.models import Word2Vec
w2v_model=Word2Vec(sentences=[text.split() for text in train_texts],vector_size=100)
import numpy as np
embedding_dim=100
embedding_matrix=np.zeros((len(word_index)+1,embedding_dim))
for word,i in word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i]=w2v_model.wv[word]
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.47467983,  0.61463583,  0.1162165 , ..., -0.61969054,
         0.03513391,  0.19062465],
       [-0.51365018,  0.66570026,  0.10935412, ..., -0.64524651,
         0.02457583,  0.19726352],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [ ]:
model=Sequential([
        Embedding(input_dim=len(word_index)+1,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
        input_length=max_sequence_length,
        trainable=False),
        Bidirectional(LSTM(128)),
        Dense(2,activation="softmax")
])

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_padded,train_labels,epochs=10,batch_size=32,validation_split=0.2)

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 72s 700ms/step - accuracy: 0.5024 - loss: 0.6950 - val_accuracy: 0.5446 - val_loss: 0.6907
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 74s 762ms/step - accuracy: 0.5186 - loss: 0.6930 - val_accuracy: 0.4566 - val_loss: 0.6990
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 75s 690ms/step - accuracy: 0.5126 - loss: 0.6931 - val_accuracy: 0.4401 - val_loss: 0.6944
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 82s 694ms/step - accuracy: 0.5294 - loss: 0.6928 - val_accuracy: 0.4541 - val_loss: 0.6968
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 82s 699ms/step - accuracy: 0.5114 - loss: 0.6926 - val_accuracy: 0.4477 - val_loss: 0.6948
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 85s 728ms/step - accuracy: 0.5060 - loss: 0.6932 - val_accuracy: 0.5230 - val_loss: 0.6932
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 77s 677ms/step - accuracy: 0.5311 - loss: 0.6928 - val_accuracy: 0.4566 - val_loss: 0.6972
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 82s 682ms/step - accuracy: 0.5201 - loss: 0.6922 - val_accu

In [ ]:
test_loss,test_accuracy=model.evaluate(test_padded,test_labels)

31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 257ms/step - accuracy: 0.5262 - loss: 0.6920


In [ ]:
test_accuracy

0.510725200176239

Stacked LSTM

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [ ]:
import random

num_samples = len(captions)
texts = []
labels = []

for idx, line in enumerate(captions):
    if random.random() < 0.5:
        texts.append(line.strip())
        labels.append(1)
    else:
        texts.append(line.strip())
        labels.append(0)
    if idx % 1000 == 0:
        print(f"Processed {idx} / {num_samples} captions...")

print("Completed processing captions.")

split_index = int(0.8 * len(texts))
train_texts = texts[:split_index]
test_texts = texts[split_index:]
train_labels = labels[:split_index]
test_labels = labels[split_index:]

print(f"Number of training samples: {len(train_texts)}")
print(f"Number of testing samples: {len(test_texts)}")
print(f"Sample training text: {train_texts[0]}")
print(f"Sample label: {train_labels[0]}")


Processed 0 / 4895 captions...
Processed 1000 / 4895 captions...
Processed 2000 / 4895 captions...
Processed 3000 / 4895 captions...
Processed 4000 / 4895 captions...
Completed processing captions.
Number of training samples: 3916
Number of testing samples: 979
Sample training text: ROCO_81826	 Model bone showing the extent of graft insertion
Sample label: 1


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences=tokenizer.texts_to_sequences(train_texts)
test_sequences=tokenizer.texts_to_sequences(test_texts)
train_sequences

[[3, 6943, 95, 73, 17, 1, 723, 2, 187, 417],
 [3, 6944, 1536, 1390, 3446, 43],
 [3, 6945, 758, 2, 844, 6946, 76, 7, 6947, 1, 188, 488],
 [3, 6948, 246, 286, 2, 1, 272, 36, 680, 6949, 6950, 4554],
 [3,
  6951,
  350,
  7,
  6952,
  4,
  6953,
  1,
  103,
  548,
  2,
  4555,
  1260,
  759,
  21,
  115,
  6,
  1,
  65,
  14,
  311,
  287,
  22,
  351,
  2,
  1,
  29,
  147,
  156,
  16,
  3447,
  6954,
  3448,
  6,
  1,
  55,
  2,
  1,
  2729,
  4556,
  6955,
  1,
  1716,
  4,
  2729,
  4556,
  9,
  5,
  1127,
  549,
  50,
  1934,
  6956,
  6957,
  6958,
  4,
  3449,
  6,
  5,
  2261,
  87,
  1537,
  30,
  1717,
  16,
  1261,
  6,
  4557,
  4,
  2262,
  2730],
 [3, 6959, 489, 2731, 6, 352, 2, 2263, 4558, 4, 6960, 74, 25],
 [3, 6961, 5, 6962, 119, 26, 260, 34, 7, 1, 29, 1935, 112, 39, 845, 112],
 [3, 6963, 6964, 6965, 312],
 [3,
  6966,
  417,
  2,
  5,
  1391,
  6,
  1,
  20,
  2732,
  122,
  2733,
  1128,
  1,
  15,
  37,
  1,
  61,
  2,
  1,
  2733,
  796,
  4,
  233,
  8,
  2264,
  7,


In [ ]:
word_index=tokenizer.word_index
max_sequence_length=max(len(seq) for seq in train_sequences)
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length
train_padded=pad_sequences(train_sequences,maxlen=max_sequence_length,padding="post")
test_padded=pad_sequences(test_sequences,maxlen=max_sequence_length,padding="post")
test_padded

array([[   3,  631, 6859, ...,    0,    0,    0],
       [   3,    1, 2823, ...,    0,    0,    0],
       [   3,  847,  379, ...,    0,    0,    0],
       ...,
       [   3,   41, 2247, ...,    0,    0,    0],
       [   3,  241,  154, ...,    0,    0,    0],
       [   3, 2124,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)
train_labels

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [ ]:
from gensim.models import Word2Vec
w2v_model=Word2Vec(sentences=[text.split() for text in train_texts],vector_size=100)
import numpy as np
embedding_dim=100
embedding_matrix=np.zeros((len(word_index)+1,embedding_dim))
for word,i in word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i]=w2v_model.wv[word]
embedding_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.76642132e-01,  7.13793814e-01,  2.89635587e-04, ...,
        -5.61798573e-01,  1.19655076e-02,  1.07116647e-01],
       [-5.28836846e-01,  7.98538387e-01, -5.24015212e-03, ...,
        -5.95117331e-01, -3.17341276e-03,  1.08534023e-01],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [ ]:
model=Sequential([
        Embedding(input_dim=len(word_index)+1,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
        input_length=max_sequence_length,
        trainable=False),
        LSTM(128, return_sequences=True),
        LSTM(64),
        Dense(2,activation="softmax")
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_padded,train_labels,epochs=10,batch_size=32,validation_split=0.2)

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 64s 620ms/step - accuracy: 0.4809 - loss: 0.6938 - val_accuracy: 0.5089 - val_loss: 0.6931
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 75s 547ms/step - accuracy: 0.4859 - loss: 0.6932 - val_accuracy: 0.5077 - val_loss: 0.6931
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 82s 544ms/step - accuracy: 0.5147 - loss: 0.6930 - val_accuracy: 0.5089 - val_loss: 0.6931
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 81s 539ms/step - accuracy: 0.4756 - loss: 0.6934 - val_accuracy: 0.5089 - val_loss: 0.6931
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 82s 544ms/step - accuracy: 0.4946 - loss: 0.6932 - val_accuracy: 0.5077 - val_loss: 0.6931
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 52s 528ms/step - accuracy: 0.4959 - loss: 0.6933 - val_accuracy: 0.5089 - val_loss: 0.6930
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 83s 537ms/step - accuracy: 0.5099 - loss: 0.6933 - val_accuracy: 0.5089 - val_loss: 0.6929
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 88s 596ms/step - accuracy: 0.5016 - loss: 0.6932 - val_accu

In [ ]:
test_loss,test_accuracy=model.evaluate(test_padded,test_labels)

31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 190ms/step - accuracy: 0.5133 - loss: 0.6979


In [ ]:
test_accuracy

0.4923391342163086

Dropout LSTM

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

captions_path = '/content/all_data/train/non-radiology/captions.txt'

with open(captions_path, 'r') as file:
    captions = file.read()


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([captions])

vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)
sequences = []
for line in captions.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        ngram_sequence = token_list[:i + 1]
        sequences.append(ngram_sequence)

print("Number of sequences:", len(sequences))

#Printing some captions
print("\nSample Captions:")
for i, line in enumerate(captions.split("\n")):
    if i >= 10:
        break
    print(f"{i + 1}: {line.strip()}")

print("\nSample Sequences:")
for i, seq in enumerate(sequences[:10]):
    print(f"{i + 1}: {seq}")


Vocabulary size: 22149
Number of sequences: 125689

Sample Captions:
1: ROCO_81826	 Model bone showing the extent of graft insertion
2: ROCO_81828	 Lip-together smile photograph
3: ROCO_81829	 Resection of superficial urothelial tissue to dissect the suture material.
4: ROCO_81830	 External appearance of the neck area revealed tiny pinkish papules.
5: ROCO_81831	 According to Junius and Kuhnt, the first drawing of disciform macular disease as seen in the eye by low magnification after removal of the anterior segment. It was called Choroido-retinitis in the region of the macula lutea. “Between the papilla and macula lutea is a whitish tract which extends downwards, becoming whiter, and terminates in a sharp point.” This atlas was published in English and German.49
6: ROCO_81832	 Significant decrease in size of pneumocephalus (subarachnoid and interventricular) (case 1).
7: ROCO_81833	 A) Gantzer muscle; B) Located posterior to the anterior interosseous nerve; C) Median nerve.
8: ROCO_81

In [ ]:
import random

num_samples = len(captions)
texts = []
labels = []

for idx, line in enumerate(captions):
    if random.random() < 0.5:
        texts.append(line.strip())
        labels.append(1)
    else:
        texts.append(line.strip())
        labels.append(0)
    if idx % 1000 == 0:
        print(f"Processed {idx} / {num_samples} captions...")

print("Completed processing captions.")

split_index = int(0.8 * len(texts))
train_texts = texts[:split_index]
test_texts = texts[split_index:]
train_labels = labels[:split_index]
test_labels = labels[split_index:]

print(f"Number of training samples: {len(train_texts)}")
print(f"Number of testing samples: {len(test_texts)}")
print(f"Sample training text: {train_texts[0]}")
print(f"Sample label: {train_labels[0]}")


Processed 0 / 4895 captions...
Processed 1000 / 4895 captions...
Processed 2000 / 4895 captions...
Processed 3000 / 4895 captions...
Processed 4000 / 4895 captions...
Completed processing captions.
Number of training samples: 3916
Number of testing samples: 979
Sample training text: ROCO_81826	 Model bone showing the extent of graft insertion
Sample label: 1


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences=tokenizer.texts_to_sequences(train_texts)
test_sequences=tokenizer.texts_to_sequences(test_texts)
train_sequences

[[3, 6943, 95, 73, 17, 1, 723, 2, 187, 417],
 [3, 6944, 1536, 1390, 3446, 43],
 [3, 6945, 758, 2, 844, 6946, 76, 7, 6947, 1, 188, 488],
 [3, 6948, 246, 286, 2, 1, 272, 36, 680, 6949, 6950, 4554],
 [3,
  6951,
  350,
  7,
  6952,
  4,
  6953,
  1,
  103,
  548,
  2,
  4555,
  1260,
  759,
  21,
  115,
  6,
  1,
  65,
  14,
  311,
  287,
  22,
  351,
  2,
  1,
  29,
  147,
  156,
  16,
  3447,
  6954,
  3448,
  6,
  1,
  55,
  2,
  1,
  2729,
  4556,
  6955,
  1,
  1716,
  4,
  2729,
  4556,
  9,
  5,
  1127,
  549,
  50,
  1934,
  6956,
  6957,
  6958,
  4,
  3449,
  6,
  5,
  2261,
  87,
  1537,
  30,
  1717,
  16,
  1261,
  6,
  4557,
  4,
  2262,
  2730],
 [3, 6959, 489, 2731, 6, 352, 2, 2263, 4558, 4, 6960, 74, 25],
 [3, 6961, 5, 6962, 119, 26, 260, 34, 7, 1, 29, 1935, 112, 39, 845, 112],
 [3, 6963, 6964, 6965, 312],
 [3,
  6966,
  417,
  2,
  5,
  1391,
  6,
  1,
  20,
  2732,
  122,
  2733,
  1128,
  1,
  15,
  37,
  1,
  61,
  2,
  1,
  2733,
  796,
  4,
  233,
  8,
  2264,
  7,


In [ ]:
word_index=tokenizer.word_index
max_sequence_length=max(len(seq) for seq in train_sequences)
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length
train_padded=pad_sequences(train_sequences,maxlen=max_sequence_length,padding="post")
test_padded=pad_sequences(test_sequences,maxlen=max_sequence_length,padding="post")
test_padded

array([[   3,  631, 6859, ...,    0,    0,    0],
       [   3,    1, 2823, ...,    0,    0,    0],
       [   3,  847,  379, ...,    0,    0,    0],
       ...,
       [   3,   41, 2247, ...,    0,    0,    0],
       [   3,  241,  154, ...,    0,    0,    0],
       [   3, 2124,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)
train_labels ##[cat, dog]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [ ]:
from gensim.models import Word2Vec
w2v_model=Word2Vec(sentences=[text.split() for text in train_texts],vector_size=100)
import numpy as np
embedding_dim=100
embedding_matrix=np.zeros((len(word_index)+1,embedding_dim))
for word,i in word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i]=w2v_model.wv[word]
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.46768659,  0.57837832, -0.0325566 , ..., -0.48211718,
        -0.07977091, -0.01884571],
       [-0.50247288,  0.62971526, -0.02632979, ..., -0.50030255,
        -0.11542598, -0.01479063],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [ ]:
model=Sequential([
        Embedding(input_dim=len(word_index)+1,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
        input_length=max_sequence_length,
        trainable=False),
        LSTM(128,dropout=0.2,),
        Dense(2,activation="softmax")
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_padded, train_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.5004 - loss: 0.6953 - val_accuracy: 0.5089 - val_loss: 0.6929
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4947 - loss: 0.6934 - val_accuracy: 0.5089 - val_loss: 0.6929
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4984 - loss: 0.6932 - val_accuracy: 0.5089 - val_loss: 0.6930
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5098 - loss: 0.7006 - val_accuracy: 0.4923 - val_loss: 0.6935
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5060 - loss: 0.6932 - val_accuracy: 0.4923 - val_loss: 0.6931
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5120 - loss: 0.6932 - val_accuracy: 0.5077 - val_loss: 0.6931
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4893 - loss: 0.6933 - val_accuracy: 0.4911 - val_loss: 0.6940
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5145 - loss: 0.6930 - val_accuracy: 0.5077 - v

In [ ]:
test_loss,test_accuracy=model.evaluate(test_padded,test_labels)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4727 - loss: 0.6930


In [ ]:
test_accuracy

0.49029621481895447

Gated Reccurent Unit-GRU

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
captions_path = '/content/all_data/train/non-radiology/captions.txt'
with open(captions_path, 'r') as file:
    captions = [line.strip() for line in file if line.strip()]
num_classes = 3
categories = np.random.choice([0, 1, 2], len(captions))
labels_mapping = {0: 'Positive', 1: 'Negative', 2: 'Neutral'}
for caption, category in zip(captions, categories):
    print(f"Caption: {caption}")
    print(f"Length: {len(caption.split())} words")
    print(f"Label: {labels_mapping[category]}")
    print("-")
print(f"Total number of captions: {len(captions)}")

Streaming output truncated to the last 5000 lines.
Length: 14 words
Label: Positive
-
Caption: ROCO_86375	 MRI and arthroscopic image of a grade I lesion in an 18-year-old male. The lesion is hardly visible in sagittal source images, coronal reconstructions with a slice thickness of 1.5 mm already miss the grade I lesion
Length: 38 words
Label: Negative
-
Caption: ROCO_86376	 Leo T. Samuels, Director of the Department of Biochemistry at the University of Utah School of Medicine in Salt Lake City.
Length: 22 words
Label: Neutral
-
Caption: ROCO_86377	 The 12-chamber IPC application.Abbreviation: IPC, intermittent pneumatic compression.
Length: 9 words
Label: Positive
-
Caption: ROCO_86378	 Range-diversity plot of the terrestrial mammals of the world.Data corresponds to a PAM of 16,420 cells of 10,000 km2, and 3,617 species. The vertical line is the proportional fill of the matrix, equal to  = 1/75.93. The curved line is obtained from equation (2) with maximum covariance = 2.03×10−3. Reg

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

captions_path = '/content/all_data/train/non-radiology/captions.txt'
with open(captions_path, 'r') as file:
    captions = [line.strip() for line in file if line.strip()]

num_classes = 3
categories = np.random.choice([0, 1, 2], len(captions))

max_vocab_size = 5000
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(captions)
sequences = tokenizer.texts_to_sequences(captions)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

categorical_labels = to_categorical(categories, num_classes=num_classes)

embedding_dim = 128
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_length),
    GRU(128, dropout=0.3, return_sequences=False),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

model.fit(
    padded_sequences, categorical_labels,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

loss, accuracy = model.evaluate(padded_sequences, categorical_labels, verbose=1)
print(f"Training Accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.3359 - loss: 1.1000 - val_accuracy: 0.3075 - val_loss: 1.1002
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.3240 - loss: 1.1007 - val_accuracy: 0.3095 - val_loss: 1.1023
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.3448 - loss: 1.0995 - val_accuracy: 0.3779 - val_loss: 1.0976
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.3573 - loss: 1.0978 - val_accuracy: 0.3882 - val_loss: 1.0972
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.3439 - loss: 1.0981 - val_accuracy: 0.3871 - val_loss: 1.0970
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.3341 - loss: 1.0986 - val_accuracy: 0.3871 - val_loss: 1.0966
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3513 - loss: 1.0977 - val_accuracy: 0.3871 - val_loss: 1.0970
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3243 - loss: 1.0989 - val_accuracy: 0.3871 - v